In [28]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import shutil
from tqdm import tqdm

In [8]:
submission_files = list(Path().glob('submission-raw-*.csv'))

In [9]:
csv = pd.read_csv(submission_files[0])

In [12]:
csv['0'][0][0] == 't'

True

In [22]:
digits = {'id': []}
digits.update({str(i): [] for i in range(10)})

for _, row in csv.iterrows():
    digits['id'].append(row['id'])
    for i, text in enumerate(row[1:]):
        value = float(re.search('tensor\\((-?\\d+\\.\\d+)', text).groups()[0])
        digits[str(i)].append(value)

In [24]:
digits = pd.DataFrame(digits)

In [25]:
digits

,id,0,1,2,3,4,5,6,7,8,9
0,2049,-1.7960,-4.2461,2.2015,-2.9521,-0.3769,2.5026,10.9224,-7.9368,1.5511,-6.0359
1,2050,-1.3242,-4.6948,-5.8026,1.0881,-2.6814,-0.4458,-4.6903,-1.6659,3.2644,8.2123
2,2051,-3.1877,-1.6604,0.3791,4.2509,-2.3908,-1.5318,-3.5207,0.3390,1.5944,-0.0219
3,2052,11.9298,-5.7249,-4.1765,-1.2835,-3.5485,-1.0389,-2.9554,0.1411,-1.8064,0.0531
4,2053,-3.6230,-3.7626,0.2950,14.2909,-4.9273,-0.7095,-7.1014,-1.3415,-2.3861,-1.9023
...,...,...,...,...,...,...,...,...,...,...,...
20475,22524,-0.6896,-1.9558,-2.7908,-2.5582,11.5616,-4.8629,-1.8971,-2.4197,-2.3809,0.9514
20476,22525,-5.8777,10.6261,-3.0872,-1.8832,-0.8319,-2.1082,-0.5281,1.9100,-0.5391,-0.9140
20477,22526,1.7049,-3.8166,-1.1599,-4.0784,-0.5299,2.5074,10.4421,-6.3549,-0.5464,-2.6908
20478,22527,-5.0506,-1.2242,0.4710,5.6553,-2.7747,-2.1885,-2.3299,-4.6113,6.1689,-2.3819


In [27]:
prep_dir = Path('prep1')

In [32]:
for submission_file in tqdm(submission_files):
    csv = pd.read_csv(submission_file)
    if type(csv['0'][0]) is str:
        digits = {'id': []}
        digits.update({str(i): [] for i in range(10)})

        for _, row in csv.iterrows():
            digits['id'].append(row['id'])
            for i, text in enumerate(row[1:]):
                value = float(re.search('tensor\\((-?\\d+\\.\\d*)', text).groups()[0])
                digits[str(i)].append(value)
        csv = pd.DataFrame(digits)
    csv.to_csv(prep_dir / submission_file, index=False)

100%|██████████| 18/18 [00:26<00:00,  1.48s/it]


In [31]:
text

"tensor(-3., device='cuda:0')"